# LangChain Tutorial with Claude

This notebook demonstrates various LangChain features including:
- Basic chat interactions
- Batch processing
- Streaming responses
- Tool usage (function calling)
- System messages and prompts
- Agents

## Setup and Installation

First, install required packages:
```bash
pip install langchain-anthropic langchain-ollama python-dotenv langchain langchain-core
```

For Ollama, make sure you have Ollama installed and running locally:
```bash
# Install Ollama from https://ollama.ai
# Then pull a model, for example:
ollama pull llama3.1
```

In [ ]:
# Import required libraries
from dotenv import load_dotenv
import os
from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama

# Load environment variables from .env file
load_dotenv('../.env')

## Initialize Model (Anthropic or Ollama)

Choose which model to use by setting the `USE_OLLAMA` variable.
- Set to `False` for Anthropic Claude (requires ANTHROPIC_API_KEY)
- Set to `True` for Ollama (requires Ollama running locally)

In [ ]:
# Configuration: Choose which model to use
USE_OLLAMA = False  # Set to True to use Ollama, False to use Anthropic

if USE_OLLAMA:
    # Initialize Ollama model
    print("Using Ollama model...")
    model = ChatOllama(
        model="llama3.1",  # or "llama2", "mistral", "codellama", etc.
        base_url=os.getenv("OLLAMA_BASE_URL", "http://localhost:11434"),
        temperature=0.7,
    )
    print(f"Model initialized: Ollama (llama3.1)")
else:
    # Initialize Anthropic Claude model
    print("Using Anthropic Claude model...")
    model = ChatAnthropic(
        model="claude-3-5-sonnet-20241022",  # or "claude-3-opus-20240229", "claude-3-haiku-20240307"
        temperature=0.7,
        max_tokens=4000,
        api_key=os.getenv("ANTHROPIC_API_KEY")
    )
    print(f"Model initialized: Claude (claude-3-5-sonnet-20241022)")

## Helper Function: Switch Between Models

Use this function to easily switch between Anthropic and Ollama models during runtime.

In [ ]:
def get_model(use_ollama=False, ollama_model="llama3.1", anthropic_model="claude-3-5-sonnet-20241022"):
    """
    Get a chat model instance.
    
    Args:
        use_ollama (bool): If True, use Ollama; if False, use Anthropic
        ollama_model (str): Ollama model name (e.g., 'llama3.1', 'mistral', 'codellama')
        anthropic_model (str): Anthropic model name (e.g., 'claude-3-5-sonnet-20241022')
    
    Returns:
        ChatModel: Initialized chat model instance
    """
    if use_ollama:
        print(f"Initializing Ollama model: {ollama_model}")
        return ChatOllama(
            model=ollama_model,
            base_url=os.getenv("OLLAMA_BASE_URL", "http://localhost:11434"),
            temperature=0.7,
        )
    else:
        print(f"Initializing Anthropic model: {anthropic_model}")
        return ChatAnthropic(
            model=anthropic_model,
            temperature=0.7,
            max_tokens=4000,
            api_key=os.getenv("ANTHROPIC_API_KEY")
        )

# Example usage:
# model = get_model(use_ollama=False)  # Use Anthropic
# model = get_model(use_ollama=True, ollama_model="llama3.1")  # Use Ollama

## Available Models

### Anthropic Models:
- `claude-3-5-sonnet-20241022` - Most capable, balanced performance
- `claude-3-opus-20240229` - Most powerful for complex tasks
- `claude-3-haiku-20240307` - Fastest, most affordable

### Popular Ollama Models:
- `llama3.1` - Meta's latest Llama model
- `llama2` - Previous generation Llama
- `mistral` - Mistral AI's model
- `codellama` - Specialized for coding
- `phi3` - Microsoft's efficient model
- `gemma` - Google's open model

Run `ollama list` in your terminal to see installed models.

## Example 1: Basic Chat Interaction

Simple question-answer interaction with Claude.

In [ ]:
# Ask a simple question
response = model.invoke("What is the capital of India?")
print(response.content)

## Example 2: Streaming Response

Stream the response token by token for a more interactive experience.

In [ ]:
# Stream a response about India
for chunk in model.stream("Tell me something interesting about India in three paragraphs"):
    print(chunk.content, end='', flush=True)

## Example 3: Batch Processing

Process multiple prompts efficiently using batch operations.

In [ ]:
# Batch process multiple queries
model.batch(
    ["Tell me a joke about cats.", "Explain the theory of relativity in simple terms."],
    config={
        "max_tokens": 500,
        "max_concurrency": 3
    }
)

## Example 4: Tool Usage (Function Calling)

Define tools that Claude can use to perform specific actions.

In [ ]:
from langchain.tools import tool

# Define a simple weather tool
@tool
def get_weather(location: str) -> str:
    """Get the weather for a location."""
    # For demonstration purposes, we'll return a dummy weather report.
    return f"The weather in {location} is sunny with a high of 75°F."

# Bind the tool to the model
model_with_tools = model.bind_tools([get_weather])

# Invoke the model with a weather query
response = model_with_tools.invoke("What's the weather like in New York City?")
print(response)

## Example 5: Tool Usage with Manual Execution

Demonstrate the complete tool calling flow:
1. User asks a question
2. Model requests a tool call
3. Tool is executed
4. Result is sent back to the model
5. Model provides final answer

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain.tools import tool

# Define a tool for temperature lookup
@tool
def get_temperature(city: str) -> str:
    """Get the current temperature for a city."""
    # Simulated weather data
    temps = {"London": "15°C", "Paris": "18°C", "Tokyo": "22°C"}
    return temps.get(city, "Temperature data not available")

# Bind tool to model
model_with_tools = model.bind_tools([get_temperature])

# Step 1: User asks a question
messages = [
    HumanMessage(content="What's the temperature in London?")
]

# Step 2: Model responds with a tool call
response = model_with_tools.invoke(messages)
print("Step 1 - AI Response:")
print(f"Content: {response.content}")
print(f"Tool Calls: {response.tool_calls}\n")

# Add AI response to messages
messages.append(response)

# Step 3: Execute the tool and create ToolMessage
if response.tool_calls:
    tool_call = response.tool_calls[0]
    
    # Execute the tool
    result = get_temperature.invoke(tool_call["args"])
    
    # Create ToolMessage with the result
    tool_message = ToolMessage(
        content=result,
        tool_call_id=tool_call["id"]
    )
    
    print("Step 2 - Tool Execution:")
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")
    print(f"Result: {result}\n")
    
    # Add ToolMessage to conversation
    messages.append(tool_message)
    
    # Step 4: Model generates final answer using tool result
    final_response = model_with_tools.invoke(messages)
    print("Step 3 - Final AI Response:")
    print(final_response.content)
    
    print("\n" + "="*60)
    print("Complete Message History:")
    for i, msg in enumerate(messages, 1):
        print(f"\n{i}. {type(msg).__name__}:")
        print(f"   {msg}")

## Example 6: Using System Messages

System messages help define the assistant's behavior and expertise.

In [ ]:
from langchain.messages import (SystemMessage,
                                HumanMessage,
                                AIMessage)

# Create a conversation with system message defining expertise
messages = [
    SystemMessage(content="You are an expert travel guide."),
    HumanMessage(content="I am planning a trip to Columbia. Can you suggest some must-visit places?"),
]

response = model.invoke(messages)
print(response.content)

In [ ]:
# Another example with Python developer expertise
system_msg = SystemMessage("""
You are senior Python Developer with 5 years of
experience in building scalable ETL pipelines using PySpark.
""")

messages = [
    system_msg,
    HumanMessage(content="Explain how to optimize PySpark jobs for large datasets.")
]

response = model.invoke(messages)
print(response.content)

## Example 7: Usage Metadata

Check token usage and other metadata from the API response.

In [ ]:
# Get usage metadata from a response
response.usage_metadata

## Example 8: LangChain Agents

Create an agent that can use tools autonomously to answer questions.

In [ ]:
from langchain.agents import create_agent

# Define a simple weather tool for the agent
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# Create an agent with the model and tools
agent = create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "What is the weather in San Francisco?"}]}
)

## Example 9: Compare Anthropic vs Ollama

Compare responses from both models for the same prompt.

In [ ]:
# Test prompt
test_prompt = "Explain what LangChain is in 2 sentences."

print("=" * 60)
print("ANTHROPIC CLAUDE RESPONSE")
print("=" * 60)
anthropic_model = get_model(use_ollama=False)
anthropic_response = anthropic_model.invoke(test_prompt)
print(anthropic_response.content)

print("\n" + "=" * 60)
print("OLLAMA RESPONSE")
print("=" * 60)
ollama_model = get_model(use_ollama=True, ollama_model="llama3.1")
ollama_response = ollama_model.invoke(test_prompt)
print(ollama_response.content)

## Example 10: Model-Specific Features

Some features may work differently between providers.

In [ ]:
# Check which model you're using
def check_model_type(model):
    if isinstance(model, ChatAnthropic):
        print("Using Anthropic Claude")
        print(f"Model: {model.model}")
        print("Features: Function calling, streaming, vision (some models)")
    elif isinstance(model, ChatOllama):
        print("Using Ollama")
        print(f"Model: {model.model}")
        print(f"Base URL: {model.base_url}")
        print("Features: Streaming, local inference, no API costs")
    else:
        print("Unknown model type")

check_model_type(model)

## Notes

- Make sure to set your `ANTHROPIC_API_KEY` in the `.env` file
- The code demonstrates various LangChain patterns with Claude
- Tool usage allows Claude to interact with external functions
- System messages help guide the model's behavior and expertise
- Streaming is useful for real-time user experience
- Batch processing helps optimize multiple queries